# Huffingtonpost

In [1]:
import requests
from requests import Session
import re
from lxml.etree import HTML
import json

# 1.Define Functions

In [2]:
browser = Session()
browser.headers.update({
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'cookie': 'rxx=1ltusvg24w7.16vrm5kn&v=1; GUCS=Abw9mMO6; GUC=AQABAQFbYcxcSUIgKwSF&s=AQAAAN9a7LVo&g=W2CBmw',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9'
})

In [3]:
def get_urls_of_articles(search_url, topic, start_page, end_page):
    prefix = "https://www.newscientist.com/subject/technology"
    article_urls = []
    for i in range(start_page, end_page+1):
        try:
            r = browser.get(search_url + topic + "/page/%s" %str(i))
            html_doc = r.text
            tree = HTML(html_doc)
            urls = tree.xpath('.//h2[@class="entry-title"]/a/@href')
            article_urls.extend(map(lambda url: prefix+url, urls))
        except AttributeError:
            pass
    return article_urls

In [4]:
def article_to_text(article_url):
    text = ""
    timestamp = ""
    try:
        r = browser.get(article_url)
        tree = HTML(r.text)
        timestamp = tree.xpath('.//p[@class="published-date"]')
        timestamp = timestamp[0].text
        text = '\n'.join([''.join(p.itertext()) for p in tree.xpath('.//div[@class="article-content"]/p')])
    except AttributeError:
            pass
    return text, timestamp

# Tech

In [5]:
search_url = "https://www.newscientist.com/subject/"
topic = "technology"
article_urls = get_urls_of_articles(search_url, topic, 1, 5)

In [6]:
cnt = 1
articles_with_timestamp = []
for article_url in article_urls:
    try:
        text, timestamp = article_to_text(article_url)  #get text of article from url
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = '\n'.join(chunk for chunk in chunks if chunk)
        text = str(text)
        text = text.replace("\n", " ")
        if (len(text.split())>=60) & (len(timestamp)>0): #get text files only if it contains more than 60 words
            articles_with_timestamp.append({"timestamp": timestamp, "article_text": text})
            cnt = cnt + 1
    except:
        pass
print("Number of articles in " + topic + ":" +str(cnt-1))

Number of articles in technology:48


In [7]:
with open('newscientist_tech_article.json', 'w') as f:
    json.dump(articles_with_timestamp, f)
f.close()